In [1]:
%load_ext autoreload 
%autoreload 2
from IPython.display import Image
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt 
import numpy as np
import time

In [4]:
import motif_processor
reload(motif_processor)

<module 'motif_processor' from 'motif_processor.pyc'>

In [114]:
import common_utils 
reload(common_utils)

<module 'common_utils' from 'common_utils.pyc'>

In [115]:
start = time.time()
CrossValProcObj = common_utils.CrossValProcessor(output_dirs = ["./HEPG2_act_1_25/", "./HEPG2_rep_1_25/", 
                                                                      "./K562_act_1_25/", "./K562_act_1_25/",
                                                                      "./LCL_act_alt_1_25/"])
print(time.time()-start)

/home/alvin/Dropbox/Lab/CNN/data/processed_cnn/HEPG2_act_mpra_dna.txt
original
/home/alvin/Dropbox/Lab/CNN/data/processed_cnn/K562_act_mpra_dna.txt
original
/home/alvin/Dropbox/Lab/CNN/data/processed_cnn/LCL_act_mpra_dna.txt
original
39.3904161453


In [116]:
import pathnet
reload(pathnet)

<module 'pathnet' from 'pathnet.pyc'>

In [117]:
X_LCL=CrossValProcObj.LCL_binObj.merged_tensor
X_RC_LCL=CrossValProcObj.LCL_binObj.merged_RC_tensor
Y_LCL=CrossValProcObj.LCL_binObj.merged_tar
test_dict = {'X': X_LCL, 'X_RC': X_RC_LCL, 'Y': Y_LCL}

In [118]:
X_HepG2=CrossValProcObj.HepG2_binObj.merged_tensor
X_RC_HepG2=CrossValProcObj.HepG2_binObj.merged_RC_tensor
Y_HepG2=CrossValProcObj.HepG2_binObj.merged_tar
HepG2_weights =  calculate_binary_weights(Y_HepG2)

In [119]:
from pathnet import SimpleCNNModel, calculate_binary_weights

In [120]:
simpleCNNobj = SimpleCNNModel(X_HepG2, X_RC_HepG2)

In [121]:
simpleCNNobj.build_model()

In [113]:
simpleCNNobj.train(X_HepG2, X_RC_HepG2, Y_HepG2, HepG2_weights[0], HepG2_weights[1])

ValueError: Error when checking model input: expected Input to have shape (None, 1, 16, 149) but got array with shape (12169, 1, 4, 150)

In [47]:
from sklearn.metrics import auc, log_loss, precision_recall_curve, roc_auc_score, roc_curve

In [50]:
Y_LCL_Pred = simpleCNNobj.model.predict([X_LCL, X_RC_LCL])
roc_auc_score(Y_LCL.astype(bool),Y_LCL_Pred )

0.66542942337993893

In [105]:
from dinucleotide_processor import SimpleDiCNNModel, process_th_dinuc_tensor

In [54]:
CrossValProcObj.LCL_binObj.merged_tensor.shape

(20870, 1, 4, 150)

In [55]:
CrossValProcObj.LCL_binObj.merged_RC_tensor.shape

(20870, 1, 4, 150)

In [57]:
CrossValProcObj.LCL_binObj.merged_RC_tensor[0,0,:,149]

array([ 1.,  0.,  0.,  0.])

In [96]:
diX_LCL= process_th_dinuc_tensor(CrossValProcObj.LCL_binObj.merged_tensor)
diX_RC_LCL= process_th_dinuc_tensor(CrossValProcObj.LCL_binObj.merged_RC_tensor)
Y_LCL=CrossValProcObj.LCL_binObj.merged_tar
test_dict = {'X': diX_LCL, 'X_RC': diX_RC_LCL, 'Y': Y_LCL}

In [101]:
diX_HepG2=process_th_dinuc_tensor(CrossValProcObj.HepG2_binObj.merged_tensor)
diX_RC_HepG2=process_th_dinuc_tensor(CrossValProcObj.HepG2_binObj.merged_RC_tensor)
Y_HepG2=CrossValProcObj.HepG2_binObj.merged_tar
HepG2_weights =  calculate_binary_weights(Y_HepG2)

In [102]:
diCNN = SimpleDiCNNModel(diX_HepG2, diX_RC_HepG2)
diCNN.build_model()

dinucleotide_processor.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation=<function ..., kernel_regularizer=<keras.reg..., filters=200, kernel_size=(16, 18))`
  self.conv_layer = Convolution2D(nb_filter=conv_filters, nb_row=16, nb_col=18, activation=relu, W_regularizer=l1_l2(L1,L2))
dinucleotide_processor.py:117: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.X_merged = merge([self.X_max_out, self.X_RC_max_out], mode=merge_mode)
dinucleotide_processor.py:128: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, activation="relu", kernel_regularizer=<keras.reg...)`
  self.X_feat_new = Dense(units, activation ='relu', W_regularizer=l1_l2(L1,L2))(self.X_feat_concat_dropout)
dinucleotide_processor.py:129: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_regularizer=<keras.re

In [103]:
diCNN.train(diX_HepG2, diX_RC_HepG2, Y_HepG2, HepG2_weights[0], HepG2_weights[1])

dinucleotide_processor.py:151: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  history = self.model.fit(x=[X, X_RC], y=y, batch_size = batch_size, nb_epoch = nb_epoch, callbacks=callbacks,validation_split=val_split, shuffle=True, class_weight={0:neg_weight, 1:pos_weight}, verbose=verbose)


In [104]:
Y_LCL_Pred = diCNN.model.predict([diX_LCL, diX_RC_LCL])
roc_auc_score(Y_LCL.astype(bool),Y_LCL_Pred )

0.67679537068432494